In [ ]:

%matplotlib auto


In [2]:
import os
import inspect
import datetime
from datetime import date, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.widgets import Button, SpanSelector

In [ ]:
class otpParser():
    
    def __init__(self):
        self.headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'currentValue', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'noIdea5']
        
    def parseFiles(self, files):
        dataFrames = [pd.read_csv(file, header=None) for file in files]
        mergedFrame = pd.concat(dataFrames)
        mergedFrame.columns = self.headers                                        
        mergedFrame = mergedFrame.reset_index(drop=True)
        self.mergedFrame = mergedFrame.sort_values(by='date')
        #return self.mergedFrame.head()

    
    def calculateAttributes(self):
        
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
#         {'expense':self.expense,'income':np.random.rand(rangeD) }
        transactions = {'expense':self.expense,'income':self.income }
        return self.timeAxis, transactions

        
    def printDf(self):
        return (self.mergedFrame.head())
    
    def calculateDates(self):
        
        start = self.mergedFrame['date'].values[0]
        end = self.mergedFrame['date'].values[-1]
        self.pandasStartDate = "{0}-{1}-{2}".format(str(start)[0:4], str(start)[4:6], str(start)[6:8])
        self.startDate = date(int(str(start)[0:4]), int(str(start)[4:6]), int(str(start)[6:8]))
        self.endDate = date(int(str(end)[0:4]), int(str(end)[4:6]), int(str(end)[6:8]))
        self.timeDelta = self.endDate - self.startDate
        #print (self.startDate, self.endDate, self.timeDelta)
        
    def separateTransactions(self):
        days = [int((self.startDate + timedelta(days=i)).strftime('%Y%m%d')) for i in range(self.timeDelta.days + 1)]
        financeAxis = np.ones(len(days))
        for idx, day in enumerate(days):

            financeAxis[idx] = self.mergedFrame.loc[self.mergedFrame['date']==day, 'sum'].sum()

        self.expense = [-figure if figure<0 else 0 for figure in financeAxis]
        self.income = [figure if figure>0 else 0 for figure in financeAxis]

        pass
        
    def selectMoney(self):
        history = self.mergedFrame['sum']
        
    def createFigures(self):
        self.calculateDates()
        self.separateTransactions()
        self.timeAxis = pd.date_range(self.pandasStartDate, periods=self.timeDelta.days+1, freq='D')
        self.viewer = financeViewer(self.timeAxis,self.charges, self.salary)


In [ ]:
class Presenter():
    
    def __init__(self, model, view, files):
        self.model = model
        self.view = view
        self.files = files
        
        self.model.parseFiles(files)
        #self.model.mergeDataFrames()
        self.timeAxis, self.transactions = self.model.calculateAttributes()

    def showUp(self):
        self.view.showPlots(self.timeAxis, self.transactions)
    
    def onResetClicked(self):
        pass
    
    def onSelect(self):
        pass

In [ ]:
files =  [file for file in os.listdir() if file.lower().endswith('csv')]
model = otpParser()
view = financeViewer()


myApp = Presenter(model, view, files)

In [ ]:
myApp.showUp()

In [ ]:
RED = '#d62728'
GREY = '#abb2b9'
GREEN = '#196f3d'
WHITE = ''
BLACK = ''
PURPLE = '#aaace2' # Color of the buttons, titles
LIGHT_GREEN = '#a9dfbf'
LIGHT_RED = '#ff9494'
LIGHT_GREY = '#d5d8dc'
width = 0.3
WIDTH = 12
G_RAT = (1 + 5 ** 0.5) / 2 # golden ratio
LABEL_ROTATION = 15 # DEGREES
DATEFROMAT = mdates.DateFormatter('%Y-%m-%d')

## TODO:
# refactorize to make it model -view- presenter
# pythonize slicing based on return value of spanSelector
# make xLabel of axis2 readable always
# add more info to show
# update, linewidth, selectionColor on axis2
# adjust colors
# rotate labels,
# fix labelformat
# adjust spacing
# generalize button layout
class financeViewer():
    
    def __init__(self):

        self.box = dict(facecolor='blue', pad=3, alpha=0.2, boxstyle="Round4,pad=0.3")
        self.testString ="""Date: {}
                            Sum: {} HUF
                            Comment:
                            Account:
                            xCoord: {}
                            yCoord: {}"""

    def createFigure(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        self.fig = plt.figure(figsize=(WIDTH, WIDTH/G_RAT),facecolor = LIGHT_GREY)

        self.gsp = gridspec.GridSpec(
                nrows = 3, ncols = 2, wspace = 0.05, hspace = 0.45,
                width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

        self.ax1 = plt.subplot(self.gsp[0,:])
        self.ax2 = plt.subplot(self.gsp[1:,0])
        self.ax3 = plt.subplot(self.gsp[1,1])
        self.ax4 = plt.subplot(self.gsp[2,1])
        

    def drawAxes(self):
        print ('{} function is called'.format(inspect.stack()[0][3]))
        for ax in [self.ax1,self.ax2,self.ax3, self.ax4]: 
            ax.set_facecolor(GREY)
        #####BIG PLOT##
        self.ax1.set_title('Whole duration',bbox=self.box)
        self.ax1.bar(self.timeAxis, self.expense, color=RED)
        self.ax1.bar(self.timeAxis, self.income, color=GREEN)
        
        self.span = SpanSelector(self.ax1, self.onselect, 'horizontal', 
                   rectprops=dict(alpha=0.3, facecolor=RED))
        
        ####ZOOM PLOT##
        self.ax2.set_title('Selected duration', bbox=self.box)
        self.ax2.bar(self.timeAxis, self.expense, width, color=RED)
        self.ax2.bar(self.timeAxis, self.income , width, color=GREEN)

        ##info plot##
        self.txt = self.ax3.text(0.1,0.5,'',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=12, color='black',
        wrap = True)
        self.ax3.set_xticks([]) 
        self.ax3.set_yticks([]) 
        self.ax3.set_title('info about the transactions', bbox=self.box)

        ### place of buttons##
        self.ax4.set_xticks([]) 
        self.ax4.set_yticks([]) 

    def on_plot_hover(self, event):
#         print ('{} function is called'.format(inspect.stack()[0][3]))
        if not event.inaxes: return
        if event.inaxes!= self.ax2: return

        for idx,bar in enumerate(self.ax2.patches):
            if bar.get_x() < event.xdata < bar.get_x() + bar.get_width():
                if bar.get_y() < event.ydata < bar.get_y() + bar.get_height(): 
                    self.ax2.patches[idx].set_edgecolor(LIGHT_RED)
                    newStr = self.testString.format('',int(bar.get_height()),  event.x, event.y, )
                    self.txt.set_text(newStr)
            else: 
                self.ax2.patches[idx].set_edgecolor(self.ax2.patches[idx].get_facecolor())
            
        self.fig.canvas.draw()

    def reset_button_on_clicked(self, mouse_event):

        ## redraw axis 2
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        self.ax2.bar(self.timeAxis, self.expense, width, color=RED)
        self.ax2.bar(self.timeAxis, self.income, width, color=GREEN)
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        self.fig.canvas.draw()

    def balanceView_button_on_clicked(self, mouse_event):
        #print ('{} function is not implemented'.format(inspect.stack()[0][3]))
        pass
    def transView_button_on_clicked(self, mouse_event):
        #print ('{} function is not implemented'.format(inspect.stack()[0][3]))
        pass
    def onselect(self, xmin, xmax):

        dayMin, dayMax = sorted((int(xmin-0.5), int(xmax+0.5)))
        ##xmin, xmax is days from zero, if Xaxis is pandas daterange

        yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
        startDate = yearZero + timedelta(days=dayMin)
        endDate = yearZero + timedelta(days=dayMax)
        st=str(startDate)[:10]
        nd=str(endDate)[:10]
        stIdx, = np.where( self.timeAxis.values==np.datetime64(st) )
        endIdx, = np.where( self.timeAxis.values==np.datetime64(nd) )
        stIdx , endIdx = stIdx[0], endIdx[0]
        zoomedTime = self.timeAxis[stIdx:endIdx]
        zoomedCharges = self.expense[stIdx:endIdx]
        zoomedSalary = self.income[stIdx:endIdx]
        
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        self.ax2.bar(zoomedTime, zoomedCharges, width, color=RED)
        self.ax2.bar(zoomedTime, zoomedSalary , width, color=GREEN)
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        self.fig.canvas.draw()
        
    def makeButtons(self):

        pos = self.ax4.get_position() # get the  position of axis ,which contains the buttons 
        rowNr, colNr = 1,3
        buttonwidth = 0.1
        buttonheight = 0.07
        Vspace = (pos.width - colNr*buttonwidth)/(colNr+1)
        Hspace = (pos.height - rowNr*buttonheight)/(rowNr+1)
        
        reset_button_ax = self.fig.add_axes([pos.x0+Vspace, pos.y0+Hspace, buttonwidth, buttonheight])
        transactionViewButon_ax = self.fig.add_axes([pos.x0+2*Vspace+buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])
        balanceViewButton_ax = self.fig.add_axes([pos.x0+3*Vspace+2*buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])

        self.reset_button = Button(reset_button_ax, 'Reset', color=PURPLE, hovercolor=RED)
        self.transactionViewButton = Button(transactionViewButon_ax, 'Transactions', color=PURPLE, hovercolor=RED)
        self.balanceViewButton = Button(balanceViewButton_ax, 'Balance', color=PURPLE, hovercolor=RED)
    
    def connectButtons(self):

        self.reset_button.on_clicked(self.reset_button_on_clicked)
        self.transactionViewButton.on_clicked(self.transView_button_on_clicked)
        self.balanceViewButton.on_clicked(self.balanceView_button_on_clicked)

    
    def showPlots(self, timeAxis, transactions):
        self.timeAxis = timeAxis
        self.income = transactions['income']
        self.expense = transactions['expense']
        self.createFigure()
        self.drawAxes()

        print ('{} function is called'.format(inspect.stack()[0][3]))
        #self.fig.canvas.mpl_connect('motion_notify_event', self.on_plot_hover) 
        self.fig.subplots_adjust(left=0.06, bottom=0.07, right=0.97, top=0.95)
        self.makeButtons()
        self.connectButtons()
        plt.setp( self.ax1.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        self.ax1.xaxis.set_major_formatter(DATEFROMAT)
        self.ax2.xaxis.set_major_formatter(DATEFROMAT)
        plt.show()

In [ ]:
# View test:
rangeD=22
datAxis = pd.date_range("1996-01-02", periods=rangeD, freq='D')
viewer = financeViewer()
viewer.showPlots(datAxis ,{'expense':np.random.rand(rangeD),'income':np.random.rand(rangeD) })

In [ ]:
files =  [file for file in os.listdir() if file.lower().endswith('csv')]
print (files)

In [ ]:
files =  [file for file in os.listdir() if file.lower().endswith('csv')]
print (files)
myOTP = otpParser(files)

In [ ]:
myOTP.createFigures()

In [ ]:
myOTP.printDf()

In [ ]:
dayMin, dayMax = 76555, 77545
##xmin, xmax is days from zero, if Xaxis is pandas daterange
print (10)
yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
print (11)
startDate = yearZero + timedelta(days=dayMin)
endDate = yearZero + timedelta(days=dayMax)

In [ ]:
# dir(viewer.ax2.patches[0])

In [7]:
def onselect(a,b):
    print (a,b)
fig = plt.figure()
G_RAT = 1.601
gsp = gridspec.GridSpec(
        nrows = 3, ncols = 2, wspace = 0.1, hspace = 0.3,
        width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

ax1 = plt.subplot(gsp[0,:])
#fig.autofmt_xdate()
ax2 = plt.subplot(gsp[1:,0])
#fig.autofmt_xdate()
ax3 = plt.subplot(gsp[1,1])
ax4 = plt.subplot(gsp[2,1])    
rangeD=22
datAxis = pd.date_range("1996-01-02", periods=rangeD, freq='D')

ax2.bar(datAxis, np.random.rand(rangeD))
ax2.bar(datAxis, np.random.rand(rangeD))
plt.setp( ax2.xaxis.get_majorticklabels(), rotation=45 )
plt.setp( ax3.xaxis.get_majorticklabels(), rotation=45 )
span = SpanSelector(ax1, onselect, 'horizontal', 
                    rectprops=dict(alpha=0.3, facecolor='red'))
myFmt = mdates.DateFormatter('%Y-333%m')
ax2.xaxis.set_major_formatter(myFmt)

plt.show()

0.0947580645161 0.788306451613
0.399193548387 0.695564516129


In [ ]:
#viewer=financeViewer(np.arange(20), np.random.rand(20), np.random.rand(20))

In [ ]:
viewer = financeViewer(np.arange(20), np.random.rand(20), np.random.rand(20))

In [ ]:
paelo=  otpParser(files)

In [ ]:
app = wx.App(False)
frame = MyForm()
frame.Show()
app.MainLoop()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def onselect(xmin, xmax):
    print (xmin, xmax)
    start_date = '0001/01/01'
    date_1 = datetime.datetime.strptime(start_date, "%Y/%m/%d")

    end_date1 = date_1 + datetime.timedelta(days=xmin)
    end_date2 = date_1 + datetime.timedelta(days=xmax)
    print (end_date1, end_date2)
timeAxis =  pd.date_range("1996-01-02", periods=45, freq='D')
fig, ax = plt.subplots()
ax.bar(timeAxis, np.random.rand(len(timeAxis)))


span = SpanSelector(ax, onselect, 'horizontal', 
                    rectprops=dict(alpha=0.3, facecolor=RED))
        

In [ ]:
tt=myOTP.timeAxis
tt

In [ ]:
vals=tt.values
vals[:3]

In [ ]:
vals.index['2017-05-03T00:00:00.000000000']
argmax(myArray)

In [ ]:
i, = np.where( tt.values==np.datetime64('2017-05-03') )
i

In [ ]:
dayMin, dayMax = sorted((int(705343), int(734343)))
##xmin, xmax is days from zero, if Xaxis is pandas daterange
print (dayMin, dayMax)
yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")

startDate = yearZero + datetime.timedelta(days=dayMin)
endDate = date_1 + datetime.timedelta(days=dayMax)
print (startDate, endDate, type(startDate))
ss=str(startDate)[:10]

print (ss)
# h=np.datetime64(startDate)
# print (h, type(h))
# k=[h,h,h,h]
# print (k)